In [ ]:
from roboflow import Roboflow
from PIL import Image, ImageDraw, UnidentifiedImageError
import PIL
import os

In [4]:
# Initialize Roboflow
rf = Roboflow(api_key="gJvPdZaOYF8eb2ctWoWS")

# Specify project and model details
project_name = "beekeeping_step1-5n131"
model_version = 10
confidence_threshold = 40
overlap_threshold = 30

# Get the project and model
project = rf.workspace().project(project_name)
model = project.version(model_version).model

# Directory where your images are stored
image_directory = "C:/Users/loicb/Pictures/Bekeeping/img/"  # Change this path to the directory where your images are stored

# List all image file names in the directory
image_file_names = [f for f in os.listdir(image_directory) if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]

# Directory for saving inference results
output_directory = "inference"
os.makedirs(output_directory, exist_ok=True)

# Perform inference in a loop
for image_file_name in image_file_names:  
    try:
        # Construct the full path to the image
        image_path = f"C:/Users/loicb/Pictures/Bekeeping/img/{image_file_name}"  # Change this path to the directory where your images are stored

        # Perform inference on the local image
        result = model.predict(image_path, confidence=confidence_threshold, overlap=overlap_threshold).json()

        # Open the original image
        original_image = Image.open(image_path)

        # Check if "objects" key is present in the result
        if "objects" in result:
            predictions = result["objects"]
        else:
            predictions = result["predictions"]

        for i,prediction in enumerate(predictions):
            # Extract relevant information
            x = int(prediction['y'])
            y = int(prediction['x'])
            width = int(prediction['width'])/2
            height = int(prediction['height'])/2
            image_path = prediction['image_path']

            # Open the original image
            original_image = Image.open(image_path)

            # Crop the region of interest
            #cropped_image = original_image.crop((left, top, right, bottom))
            cropped_image = original_image.crop((y - width, x - height, y + width, x + height))

            # Save the cropped image to a new file
            crop_file_name = image_file_name.split('.')[0]
            output_file_name = f"{output_directory}/{crop_file_name}_{i}.jpg"
            cropped_image.save(output_file_name)
            #print(f"Cropped image saved: {output_file_name}")
    except UnidentifiedImageError:
        print(f"Skipping {image_file_name} due to UnidentifiedImageError.")

print(f"Saved detection frame at: {os.path.join(output_directory, f'detection_frame_{image_file_name}')}")  

# End of the loop


loading Roboflow workspace...
loading Roboflow project...


HTTPError: 403 Client Error: Forbidden for url: https://detect.roboflow.com/beekeeping_step1-5n131/9?api_key=gJvPdZaOYF8eb2ctWoWS&name=YOUR_IMAGE.jpg&overlap=30&confidence=40&stroke=1&labels=false&format=json